<a href="https://colab.research.google.com/github/Deep-Poets-Society/Deep-Comedy/blob/main/generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colab Setup

import sys
import os

%cd /content
!rm -rf Deep-Comedy &> /dev/null
!git clone https://github.com/Deep-Poets-Society/Deep-Comedy &> /dev/null
%cd Deep-Comedy
sys.path.append(os.getcwd())
!git clone https://gitlab.com/sasso-effe/deep-comedy-checkpoints.git &> /dev/null

!pip install tensorflow &> /dev/null
!pip install tensorflow_text &> /dev/null

/content
/content/Deep-Comedy


In [2]:
from tokenizer import Tokenizer
import tensorflow as tf
import time
import numpy as np
import re
from matplotlib import pyplot as plt
from utils import load_dataset_for_gen, positional_encoding, loss_function, accuracy_function, plot_accuracy
from models import Transformer, create_masks
from schedules import CustomSchedule
from preprocessing import CESURA, SPACE, SYL, NEW_LINE
import random

In [3]:
dataset = load_dataset_for_gen()
train = dataset['train']
val = dataset['val']

In [4]:
tokenizer = Tokenizer(['[START]', '[END]', f'{SPACE}', f'{NEW_LINE}'], 'res/vocab_gen.txt')

In [5]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [6]:
def tokenize_pairs(X, y):
    X = tokenizer.tokenize(X)
    # Convert from ragged to dense, padding with zeros.
    X = X.to_tensor()

    y = tokenizer.tokenize(y)
    # Convert from ragged to dense, padding with zeros.
    y = y.to_tensor()

    return X, y


def make_batches(ds):
    return (ds
            .cache()
            .shuffle(BUFFER_SIZE)
            .batch(BATCH_SIZE)
            .map(tokenize_pairs, num_parallel_calls=tf.data.AUTOTUNE)
            .prefetch(tf.data.AUTOTUNE)
            )

In [7]:
train_batches = make_batches(train)
val_batches = make_batches(val)

Hyperparameters

In [8]:
num_layers = 6
d_model = 512
dff = 2048
num_heads = 8
dropout_rate = 0.1

Set Adam Optimizer

In [9]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

Loss and metrics

In [10]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

val_loss = tf.keras.metrics.Mean(name='val_loss')
val_accuracy = tf.keras.metrics.Mean(name='val_accuracy')

Training and checkpoint


In [11]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=tokenizer.get_vocab_size().numpy(),
    target_vocab_size=tokenizer.get_vocab_size().numpy(),
    pe_input=1000,
    pe_target=1000,
    rate=dropout_rate)

In [12]:
restore_checkpoint = True

In [14]:
checkpoint_path = "./deep-comedy-checkpoints/gen"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if restore_checkpoint and ckpt_manager.latest_checkpoint:
    checkpoint = ckpt_manager.checkpoints[0]
    ckpt.restore(checkpoint)
    print('Checkpoint restored!!')
    print(checkpoint)

Checkpoint restored!!
./deep-comedy-checkpoints/gen/ckpt-1


In [15]:
EPOCHS = 50

In [16]:
# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]


@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]

  with tf.GradientTape() as tape:
    predictions, _ = transformer([inp, tar_inp],
                                 training = True)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

  train_loss(loss)
  train_accuracy(accuracy_function(tar_real, predictions))

In [17]:
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

In [18]:
if not restore_checkpoint:
  for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
    train_accuracy.reset_states()
    val_loss.reset_states()
    val_accuracy.reset_states()

    # inp -> X, tar -> y
    for (batch, entry) in enumerate(train_batches):
      inp = entry[0]
      tar = entry[1]
      train_step(inp, tar)

      #if batch % 50 == 0:
      #  print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

    if (epoch + 1) % 50 == 0:
      ckpt_save_path = ckpt_manager.save()
      print(f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')
    
    for val_entry in val_batches:
      val_inp = val_entry[0]
      val_tar = val_entry[1]
      val_tar_inp = val_tar[:, :-1]
      val_tar_real = val_tar[:, 1:]
      with tf.GradientTape() as tape:
        predictions, _ = transformer([val_inp, val_tar_inp],
                                  training = False)
      loss = loss_function(val_tar_real, predictions)
      val_loss(loss)
      val_accuracy(accuracy_function(val_tar_real, predictions))
    
    print(f'Epoch {epoch + 1} Train loss {train_loss.result():.4f} Train accuracy {train_accuracy.result():.4f}')
    print(f'Epoch {epoch + 1} Validation loss {val_loss.result():.4f} Validation accuracy {val_accuracy.result():.4f}')

    train_losses.append(train_loss.result().numpy())
    train_accuracies.append(train_accuracy.result().numpy())
    val_losses.append(val_loss.result().numpy())
    val_accuracies.append(val_accuracy.result().numpy())

    print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')

Evaluate

In [19]:
def add_randomness(arr, r):
  rand_arr = [1.0 + random.uniform(-r, r) for _ in arr]
  rand_arr = np.array(rand_arr, dtype = float)
  return np.multiply(arr, rand_arr)

In [20]:
def beam_search(line, max_length=1000, beam_width=10, random_factor=0.1):
  '''
  Parameters
  ----------
  line : str
      The input line
  max_length : int, optional
      The max length in tokens of the output string (default is 1000)
  beam_wodth : int, optional
      The number of beams (default is 10)
  random_factor : float, optional
      To get a non-deterministic solution
  '''
  line = tf.convert_to_tensor([line])
  encoder_input = tokenizer.tokenize(line).to_tensor()

  start, end = tokenizer.tokenize([''])[0]
  output = tf.convert_to_tensor([start])
  output = tf.expand_dims(output, 0)

  # predictions.shape == (batch_size, seq_len, vocab_size)
  predictions, attention_weights = transformer.call((encoder_input, output),
                                              False)
  predictions = tf.nn.softmax(predictions, axis=-1)
  # select the last word from the seq_len dimension
  predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)
  predictions = add_randomness(predictions.numpy()[0][0], random_factor) 
  predictions = np.log(predictions)
  tokens = np.argpartition(predictions, -beam_width)[-beam_width:]
  probabilities = predictions[tokens]
  beams = []
  for token in tokens:
    token = tf.convert_to_tensor([token], dtype=tf.int64)
    token = tf.expand_dims(token, 0)
    beams.append(tf.concat([output, token], axis=-1))

  for j in range(max_length-1):
    n_ended = 0
    candidates = []
    candidates_probs = []
    is_all_ended = True
    for k, beam in enumerate(beams):
      if beam.numpy()[0][-1] != end:
        is_all_ended = False
        # predictions.shape == (batch_size, seq_len, vocab_size)
        predictions, attention_weights = transformer.call((encoder_input, beam),
                                                    False)
        predictions = tf.nn.softmax(predictions, axis=-1)
        # select the last word from the seq_len dimension
        predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)
        predictions = add_randomness(predictions.numpy()[0][0], random_factor) 
        predictions = np.log(predictions)
        indeces = np.argpartition(predictions, -beam_width)[-beam_width:]
        values = predictions[indeces] + probabilities[k]
        for index, prob in zip(indeces, values):
          index = tf.convert_to_tensor([index], dtype=tf.int64)
          index = tf.expand_dims(index, 0)
          candidates.append(tf.concat([beam, index], axis=-1))
          candidates_probs.append(prob)
      else:
        n_ended += 1
    if is_all_ended:
      break
    best_probs = np.argpartition(candidates_probs, -(beam_width-n_ended))[-(beam_width-n_ended):]
    counter = 0
    for i in range(len(beams)):
      if beams[i].numpy()[0][-1] != end:
        beams[i] = candidates[best_probs[counter]]
        probabilities[i] = candidates_probs[best_probs[counter]]
        counter += 1

  text = tokenizer.detokenize(beams[0])[0]
  tokens = tokenizer.lookup(beams[0])[0]
  return text, tokens, attention_weights
  

In [21]:
def top_k(line, max_length=1000, k=1, r=1.0):
  '''
  Parameters
  ----------
  line : str
      The input line
  max_length : int, optional
      The max length in tokens of the output string (default is 1000)
  k : int, optional
      The parameter k of top-k sampling, with 1 the function behaves as a
      standard greedy selection (default is 1)
  r : float, optional
      sampling is applied with probability r, so with r=0.0 the function behaves
      as a standard greedy selection, with r=1.0 as a standard top-k sampling
      (default is 1.0)
  '''
  line = tf.convert_to_tensor([line])
  line = tokenizer.tokenize(line).to_tensor()

  encoder_input = line

  start, end = tokenizer.tokenize([''])[0]
  output = tf.convert_to_tensor([start])
  output = tf.expand_dims(output, 0)

  for i in range(max_length):
    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, attention_weights = transformer.call((encoder_input, output),
                                                 False)
    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)
    if i > 0 and random.random() < r:
      predictions = tf.nn.softmax(predictions, axis=-1)
      top = tf.math.top_k(predictions.numpy()[0][0], k=k)
      predicted_id = random.choices(top.indices.numpy(), weights=top.values.numpy(), k=1)
      predicted_id = tf.convert_to_tensor([predicted_id], dtype=tf.int64)
    else:
      predicted_id = tf.argmax(predictions, axis=-1)
    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)
    # return the result if the predicted_id is equal to the end token
    if predicted_id == end:
      break

  # output.shape (1, tokens)
  text = tokenizer.detokenize(output)[0]
  tokens = tokenizer.lookup(output)[0]
  return text, tokens, attention_weights

In [22]:
def print_prediction(x, true=None, add_spaces=True):
  if true is None:
    true = 'Not provided\n'

  if add_spaces:
    x = re.sub(' ', ' S ', x)
  
  predicted, _, _ = beam_search(x)
  predicted = predicted.numpy().decode('utf-8')
  x = re.sub(' S ', ' ', x)
  print(f'Original:\n{x}\n')
  print(f'Predicted:\n{predicted}\n')
  print(f'True:\n{true}\n')
  print('#'*50)

In [31]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

test = dataset['test']
pick = 2
print("seed number:", pick)
tercets = random.randint(5, 5)
for i, (x, y) in enumerate(test.batch(1)):
  if i == pick:
    x = x.numpy()[0].decode('utf-8')
    break
print('seed:\n' + re.sub(' S ', ' ', re.sub(' N ', '\n ', x)) + '\n')
print('---generated:---\n')
for i in range(tercets):
  predicted, _, _ = beam_search(x, beam_width=5, random_factor=0)
  predicted = predicted.numpy().decode('utf-8')
  print(predicted + '\n')
  x = re.sub('[|$]', '', re.sub(r'\n', ' N ', re.sub(' ', ' S ', predicted)))

seed number: 2
seed:
 i’ fui colui che la ghisolabella
 condussi a far la voglia del marchese
 come che suoni la sconcia novella 

---generated:---

 |ben |m’ ac|cor|si |ch’ el|li e|ra $da |ciel |mes|so
|ch’ io |vol|gea i |cad|di e $ri|ma|se e ’l |vol|to
|non |per |ve|der |non |ei $com’ |io |ti |mes|so

|co|sì |l’ ae|re |vi|cin $qui|vi |si |mi|ra
|qui|vi è il |gran |d’ o|gne $par|te u|dir |quin|ta
|co|sì |ri|spuo|se a |me $che |ti |ri|mi|ra

|e |l’ al|tro |dis|se |quel $che |tu |hai |guar|do
|ri|tro|ve|rai |co|me a $quel|l’ uom |ti|ra
|co|me an|cor |ti |sa|reb|be in $ma|ra|vi|glia

 |ben |ma|e|stro |mio $dis|s’ io |ch’ io |vi|di
|ed |el |s’ ac|cor|s’ io |e|ra $sì |com’ |el|l’ ac|cor|to
|ch’ io |di|co |co|lui $che |fui |tut|ti |ti|ri

 |ben |ma|e|stro |mio $dis|s’ io io |ch’ io |vi|di
|ed |el |s’ ac|cor|s’ io |ed $ed |el |s’ io |vol|to
|vol|si a |lo ’m|pe|di|re e $quel |che |tu |mi |di

